In [1]:
import os
import v20
import pandas as pd
import numpy as np
import json
import base64
import requests
import logging
import time
import os
import mplfinance as mpf
import openai
import base64
import requests
import logging
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
from oandapyV20.endpoints.orders import OrderCreate
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20.endpoints.orders import OrderCreate
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
from oandapyV20.exceptions import V20Error


# Load environment variables
load_dotenv()

# OANDA API configuration
access_token = os.getenv('OANDA_API_TOKEN')
account_id = os.getenv('OANDA_ACCOUNT_ID')
api = API(access_token=access_token, environment="practice")

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oandapyV20").setLevel(logging.WARNING)

# Parameters for real-time data fetching and processing
granularity = 'M5'
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '5 minutes'
window_size = 576  # Adjust window size for detecting single patterns
step_size = 5      # Adjust step size accordingly

def fetch_forex_data(from_date, to_date, granularity, instrument):
    logging.info(f"Fetching forex data from {from_date} to {to_date} with granularity {granularity} for instrument {instrument}")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = api.request(request)
            if response:
                for candle in response.get('candles'):
                    time = candle.get('time').split('.')[0] + 'Z'
                    rec = {
                        'time': time,
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df

def calculate_rsi(data, length=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    fast_ema = data.ewm(span=fast_period, adjust=False).mean()
    slow_ema = data.ewm(span=slow_period, adjust=False).mean()
    macd = fast_ema - slow_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal

def calculate_bollinger_bands(data, window=20, no_of_std=2):
    rolling_mean = data.rolling(window).mean()
    rolling_std = data.rolling(window).std()
    upper_band = rolling_mean + (rolling_std * no_of_std)
    lower_band = rolling_mean - (rolling_std * no_of_std)
    return rolling_mean, upper_band, lower_band

def calculate_fibonacci_retracement(data):
    max_price = data['high'].max()
    min_price = data['low'].min()
    diff = max_price - min_price
    levels = [max_price - diff * r for r in [0.236, 0.382, 0.5, 0.618, 0.786]]
    return levels

def calculate_volume_profile(data):
    price = data['close']
    volume = data['volume']
    profile = pd.concat([price, volume], axis=1)
    profile.columns = ['price', 'volume']
    profile = profile.groupby('price').sum()
    return profile

def calculate_ichimoku(data):
    high_9 = data['high'].rolling(window=9).max()
    low_9 = data['low'].rolling(window=9).min()
    high_26 = data['high'].rolling(window=26).max()
    low_26 = data['low'].rolling(window=26).min()
    high_52 = data['high'].rolling(window=52).max()
    low_52 = data['low'].rolling(window=52).min()
    
    tenkan_sen = (high_9 + low_9) / 2
    kijun_sen = (high_26 + low_26) / 2
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)
    senkou_span_b = ((high_52 + low_52) / 2).shift(26)
    chikou_span = data['close'].shift(-26)
    
    return tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b, chikou_span

def plot_candlestick_chart(df, filename, width=1200, height=800, order_details=None):
    logging.info(f"Plotting candlestick chart to {filename}")
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--', figcolor='white', facecolor='white')
    
    addplots = []
    if 'RSI' in df.columns:
        addplots.append(mpf.make_addplot(df['RSI'], panel=1, color='blue', secondary_y=False, ylabel='RSI'))
    if 'EMA' in df.columns:
        addplots.append(mpf.make_addplot(df['EMA'], color='orange'))
    if 'MACD' in df.columns and 'MACD_Signal' in df.columns:
        addplots.append(mpf.make_addplot(df['MACD'], panel=2, color='purple', secondary_y=False, ylabel='MACD'))
        addplots.append(mpf.make_addplot(df['MACD_Signal'], panel=2, color='red'))
    if 'Bollinger_Mid' in df.columns and 'Bollinger_Upper' in df.columns and 'Bollinger_Lower' in df.columns:
        addplots.append(mpf.make_addplot(df['Bollinger_Mid'], color='blue'))
        addplots.append(mpf.make_addplot(df['Bollinger_Upper'], color='red'))
        addplots.append(mpf.make_addplot(df['Bollinger_Lower'], color='red'))
    if 'Fibonacci' in df.columns:
        for level in df['Fibonacci'].dropna().unique():
            addplots.append(mpf.make_addplot([level] * len(df), color='magenta', linestyle='--'))
    
    fig, axes = mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, figsize=(width / 100, height / 100), returnfig=True)
    
    if order_details:
        ax = axes[0]  # Main candlestick plot
        entry_price = order_details['entry_price']
        take_profit = order_details['take_profit']
        stop_loss = order_details['stop_loss']
        ax.axhline(entry_price, color='blue', linestyle='--', linewidth=2, label='Entry Price')
        ax.axhline(take_profit, color='green', linestyle='--', linewidth=2, label='Take Profit')
        ax.axhline(stop_loss, color='red', linestyle='--', linewidth=2, label='Stop Loss')
        ax.legend()
    
    fig.savefig(filename, bbox_inches='tight')
    plt.close(fig)

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_chart_with_gpt4o(chart_image_path, indicators_data):
    logging.info("Sending data to OpenAI API for analysis")
    base64_image = encode_image(chart_image_path)
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    
    # Ensure all keys in indicators_data are strings
    def convert_keys_to_str(data):
        if isinstance(data, dict):
            return {str(k): convert_keys_to_str(v) for k, v in data.items()}
        elif isinstance(data, list):
            return [convert_keys_to_str(item) for item in data]
        else:
            return data

    indicators_data_str_keys = convert_keys_to_str(indicators_data)

    prompt_content = {
        "content": "Please analyze the following candlestick chart for EUR/USD and provide a detailed analysis, including detected patterns, predictions, "
                   "and potential trading signals. The chart includes various indicators such as RSI, EMA, MACD, and Bollinger Bands.\n\n"
                   "Provide the analysis in JSON format with the following structure:\n\n"
                   "{\n"
                   "    \"predictions\": [\n"
                   "        {\n"
                   "            \"timeframe\": \"# minutes\",\n"
                   "            \"patterns\": [\n"
                   "                {\"id\": #, \"pattern_detected\": #, \"pattern_name\": \"Pattern Name\", \"confidence_percentage\": ##}\n"
                   "            ],\n"
                   "            \"action\": \"####\",\n"
                   "            \"entry_price\": #.####,\n"
                   "            \"take_profit\": #.####,\n"
                   "            \"stop_loss\": #.####,\n"
                   "            \"deadline_date\": \"####-##-##T##:##:##Z\"\n"
                   "        }\n"
                   "    ]\n"
                   "}\n\n",
        "image": f"data:image/png;base64,{base64_image}",
        "indicators_data": indicators_data_str_keys  # Use the converted data with string keys
    }
    
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": json.dumps(prompt_content)
            }
        ],
        "max_tokens": 3000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    try:
        response_data = response.json()
        return response_data
    except json.JSONDecodeError:
        logging.error("Failed to decode JSON response from OpenAI API")
        return None

def extract_and_place_order(response_data):
    if not response_data or "choices" not in response_data:
        logging.error("Invalid response data")
        return None

    # Extract content from response
    content = response_data["choices"][0]["message"]["content"]
    
    # Remove the markdown and parse the JSON content
    start_index = content.find('{')
    end_index = content.rfind('}') + 1
    json_content = content[start_index:end_index]
    
    try:
        analysis = json.loads(json_content)
    except json.JSONDecodeError as e:
        logging.error(f"Failed to parse JSON content: {e}")
        return None
    
    # Extract the prediction data
    predictions = analysis.get("predictions", [])
    
    # Loop through predictions and check the confidence
    for prediction in predictions:
        for pattern in prediction.get("patterns", []):
            confidence = pattern.get("confidence_percentage", 0)
            if confidence > 70:
                # Extract order details
                action = prediction.get("action", "")
                entry_price = prediction.get("entry_price", 0.0)
                take_profit = prediction.get("take_profit", 0.0)
                stop_loss = prediction.get("stop_loss", 0.0)
                deadline_date = prediction.get("deadline_date", "")
                
                # Log the order details
                logging.info(f"Order Details - Action: {action}, Entry Price: {entry_price}, Take Profit: {take_profit}, Stop Loss: {stop_loss}")
                
                # Place the order
                order_details = {
                    'action': action,
                    'entry_price': entry_price,
                    'take_profit': take_profit,
                    'stop_loss': stop_loss,
                    'deadline_date': deadline_date
                }
                response = place_order(order_details)
                
                # Check for order cancellation and log the reason
                if 'orderCancelTransaction' in response:
                    logging.info(f"Order {response['orderCancelTransaction']['orderID']} was canceled: {response['orderCancelTransaction']['reason']}")
                return order_details  # Return the order details to plot on the chart

    logging.info("No patterns with confidence > 70% found")
    return None

def place_order(order_details):
    instrument = "EUR_USD"
    
    # Prepare order data
    mkt_order = MarketOrderRequest(
        instrument=instrument,
        units=-10000 if order_details['action'].upper() == 'SELL' else 10000,
        takeProfitOnFill=TakeProfitDetails(price=order_details['take_profit']).data,
        stopLossOnFill=StopLossDetails(price=order_details['stop_loss']).data
    )
    
    # Create the OrderCreate request
    r = orders.OrderCreate(accountID=account_id, data=mkt_order.data)
    try:
        # Submit the order request
        response = api.request(r)
        logging.info(f"Order placed successfully: {response}")
        return response
    except V20Error as e:
        logging.error(f"Error placing order: {e}")
        return {"error": str(e)}

# Define time frame for the chart
filename = "normal_chart.png"
width, height = 700, 500

# Main loop for real-time data fetching and analysis
while True:
    # Fetch data for the specified time frame
    start_time = (datetime.now(timezone.utc) - timedelta(days=2)).strftime('%Y-%m-%dT%H:%M:%SZ')  # Reduced to last 2 days to minimize data
    end_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')

    prices = fetch_forex_data(start_time, end_time, granularity, instrument)

    # Calculate additional indicators (RSI, EMA, MACD, Bollinger Bands, Fibonacci Retracement, Volume Profile, Ichimoku Cloud)
    prices['RSI'] = calculate_rsi(prices['close'])
    prices['EMA'] = prices['close'].ewm(span=20, adjust=False).mean()
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])
    prices['Fibonacci'] = pd.Series(calculate_fibonacci_retracement(prices))
    volume_profile = calculate_volume_profile(prices)
    ichimoku = calculate_ichimoku(prices)

    # Add Ichimoku Cloud data to the DataFrame
    prices['Tenkan_Sen'], prices['Kijun_Sen'], prices['Senkou_Span_A'], prices['Senkou_Span_B'], prices['Chikou_Span'] = ichimoku

    # Plot the candlestick chart
    plot_candlestick_chart(prices, filename, width=width, height=height)

    # Collect indicators data (Optimize by limiting to relevant indicators and reducing size)
    indicators_data = prices.tail(100)[['close', 'RSI', 'EMA', 'MACD', 'MACD_Signal', 'Bollinger_Mid', 'Bollinger_Upper', 'Bollinger_Lower']].to_dict()

    # Send data to OpenAI API
    analysis_result = analyze_chart_with_gpt4o(filename, indicators_data)
    logging.info(f"OpenAI API Analysis Result: {json.dumps(analysis_result, indent=4)}")

    # Extract data from the response and place an order
    order_details = extract_and_place_order(analysis_result)
    
    # Plot the chart again with order details
    if order_details:
        plot_candlestick_chart(prices, filename, width=width, height=height, order_details=order_details)

    # Sleep for 5 minutes before fetching new data
    logging.info("Waiting for 5 minutes before next run...")
    time.sleep(300)


INFO:root:Fetching forex data from 2024-05-30T03:40:29Z to 2024-06-01T03:40:29Z with granularity M5 for instrument EUR_USD
INFO:root:Plotting candlestick chart to normal_chart.png
INFO:root:Sending data to OpenAI API for analysis
INFO:root:OpenAI API Analysis Result: {
    "id": "chatcmpl-9V9taGffATgITq2X8pBv9E0k3vVfg",
    "object": "chat.completion",
    "created": 1717213234,
    "model": "gpt-4o-2024-05-13",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "```json\n{\n    \"predictions\": [\n        {\n            \"timeframe\": \"5 minutes\",\n            \"patterns\": [\n                {\"id\": 1, \"pattern_detected\": true, \"pattern_name\": \"RSI Overbought\", \"confidence_percentage\": 80},\n                {\"id\": 2, \"pattern_detected\": true, \"pattern_name\": \"MACD Bullish Crossover\", \"confidence_percentage\": 75},\n                {\"id\": 3, \"pattern_detected\": true, \"patt